# Install / Import / Config

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
import edurel.utils.llm as llmu
import edurel.utils.duckdb as ddbu
import edurel.utils.mermaid as mmu

from edurel.widgets.mermaid_viz import MermaidVisualizer

BASE_DIR = "/home/basis/work/github/edurel"
DB_DIR = f"{BASE_DIR}/databases/"

# Database

In [8]:
con = ddbu.mem_con(DB_DIR + "db-hvs")
schema_plain = ddbu.schema(con)
schema_yaml = ddbu.schema_yaml(con)
schema_mermaid = mmu.schema_to_mermaid(schema_yaml)

In [ ]:
# print(ddbu.schema(con))
# con.close()

In [9]:
viz = MermaidVisualizer(schema_mermaid, width="2000px", height="1000px")
viz.display() 

# LLM

In [6]:
deepseek32_llm_stats = llmu.chat_llm_stats(llmu.DEEPSEEK32)
gemini3pro_llm_stats = llmu.chat_llm_stats(llmu.GEMINI3PRO)
glm46_llm_stats = llmu.chat_llm_stats(llmu.GLM46)
gpt41mini_llm_stats = llmu.chat_llm_stats(llmu.GPT41MINI)
opus41_llm_stats = llmu.chat_llm_stats(llmu.OPUS41)

## Q1

In [ ]:
question = """
erstelle eine SQL-abfrage, die folgende spalten ausgibt:
- pid, name aus tabelle person
- matrnr aus tabelle studentin
sortiert nach pid
"""

### DEEPSEEK32

In [ ]:
sql = llmu.chat_text_to_sql(deepseek32_llm_stats, schema_plain, question)
print(sql)

In [ ]:
ddbu.sql_print(con, sql)

### GEMINI3PRO3

In [20]:
sql = llmu.chat_text_to_sql(gemini3pro_llm_stats, schema_plain, question)
print(sql)


SELECT
  t1.pid,
  t1.name,
  t3.matrnr
FROM person AS t1
JOIN personenrolle AS t2
  ON t1.pid = t2.person_pid
JOIN studentin AS t3
  ON t2.prid = t3.prid
ORDER BY
  t1.pid;



In [21]:
ddbu.sql_print(con, sql)

┌───────┬────────────────┬─────────┐
│  pid  │      name      │ matrnr  │
│ int32 │    varchar     │ varchar │
├───────┼────────────────┼─────────┤
│     5 │ Max Mustermann │ 1001    │
│     6 │ Lisa Wagner    │ 1002    │
│     7 │ Tom Fischer    │ 1003    │
│     8 │ Sarah Meyer    │ 1004    │
│     9 │ Jan Becker     │ 1005    │
│    10 │ Nina Schulz    │ 1006    │
└───────┴────────────────┴─────────┘



### GLM46

In [ ]:
sql = llmu.chat_text_to_sql(glm46_llm_stats, schema_plain, question)
print(sql)


SELECT p.pid, p.name, s.matrnr
FROM person p
JOIN personenrolle pr ON p.pid = pr.person_pid
JOIN studentin s ON pr.prid = s.prid
ORDER BY p.pid;



In [14]:
ddbu.sql_print(con, sql)

┌───────┬────────────────┬─────────┐
│  pid  │      name      │ matrnr  │
│ int32 │    varchar     │ varchar │
├───────┼────────────────┼─────────┤
│     5 │ Max Mustermann │ 1001    │
│     6 │ Lisa Wagner    │ 1002    │
│     7 │ Tom Fischer    │ 1003    │
│     8 │ Sarah Meyer    │ 1004    │
│     9 │ Jan Becker     │ 1005    │
│    10 │ Nina Schulz    │ 1006    │
└───────┴────────────────┴─────────┘



### GPT41MINI

In [ ]:
sql = llmu.chat_text_to_sql(gpt41mini_llm_stats, schema_plain, question)
print(sql)


SELECT p.pid, p.name, s.matrnr
FROM person p
JOIN studentin s ON p.pid = s.prid
ORDER BY p.pid;



In [22]:
ddbu.sql_print(con, sql)

┌───────┬────────────────┬─────────┐
│  pid  │      name      │ matrnr  │
│ int32 │    varchar     │ varchar │
├───────┼────────────────┼─────────┤
│     5 │ Max Mustermann │ 1001    │
│     6 │ Lisa Wagner    │ 1002    │
│     7 │ Tom Fischer    │ 1003    │
│     8 │ Sarah Meyer    │ 1004    │
│     9 │ Jan Becker     │ 1005    │
│    10 │ Nina Schulz    │ 1006    │
└───────┴────────────────┴─────────┘



### OPUS41

In [ ]:
sql = llmu.chat_text_to_sql(opus41_llm_stats, schema_plain, question)
print(sql)


SELECT p.pid, p.name, s.matrnr
FROM person p
JOIN personenrolle pr ON p.pid = pr.person_pid
JOIN studentin s ON pr.prid = s.prid
ORDER BY p.pid;



In [11]:
ddbu.sql_print(con, sql)

┌───────┬────────────────┬─────────┐
│  pid  │      name      │ matrnr  │
│ int32 │    varchar     │ varchar │
├───────┼────────────────┼─────────┤
│     5 │ Max Mustermann │ 1001    │
│     6 │ Lisa Wagner    │ 1002    │
│     7 │ Tom Fischer    │ 1003    │
│     8 │ Sarah Meyer    │ 1004    │
│     9 │ Jan Becker     │ 1005    │
│    10 │ Nina Schulz    │ 1006    │
└───────┴────────────────┴─────────┘



## Q2

In [7]:
question = """
erstelle eine SQL-abfrage, die folgende spalten ausgibt:
- name aus tabelle person
- matrnr aus tabelle studentin
- raumnr aus tabelle raum
für die studentin mit matrnr 1001
und die räume, in den sie kurse belegt hat
sortiert nach raumnr
"""

### GLM46

In [25]:
sql = llmu.chat_text_to_sql(glm46_llm_stats, schema_plain, question)
print(sql)


SELECT p.name, s.matrnr, r.raumnr
FROM studentin s
JOIN belegung b ON s.prid = b.studentin_prid
JOIN kursbt kb ON b.kursbt_kbtid = kb.kbtid
JOIN raum r ON kb.raum_rid = r.rid
JOIN personenrolle pr ON s.prid = pr.prid
JOIN person p ON pr.person_pid = p.pid
WHERE s.matrnr = '1001'
ORDER BY r.raumnr;



In [ ]:
ddbu.sql_print(con, sql)

┌────────────────┬─────────┬─────────┐
│      name      │ matrnr  │ raumnr  │
│    varchar     │ varchar │ varchar │
├────────────────┼─────────┼─────────┤
│ Max Mustermann │ 1001    │ A101    │
│ Max Mustermann │ 1001    │ A102    │
│ Max Mustermann │ 1001    │ A102    │
│ Max Mustermann │ 1001    │ B201    │
│ Max Mustermann │ 1001    │ B201    │
│ Max Mustermann │ 1001    │ C301    │
│ Max Mustermann │ 1001    │ C301    │
│ Max Mustermann │ 1001    │ D401    │
│ Max Mustermann │ 1001    │ D401    │
│ Max Mustermann │ 1001    │ E501    │
├────────────────┴─────────┴─────────┤
│ 10 rows                  3 columns │
└────────────────────────────────────┘



## Q3

In [10]:
question = """
erstelle eine SQL-abfrage, die folgende spalten ausgibt:
- name aus tabelle person
- matrnr aus tabelle studentin
für alle studenten die im sommersemester 2023 kurse belegt haben,
die am montag stattfinden
"""

### GLM46

In [11]:
sql = llmu.chat_text_to_sql(glm46_llm_stats, schema_plain, question)
print(sql)


SELECT p.name, s.matrnr
FROM person p
JOIN personenrolle pr ON p.pid = pr.person_pid
JOIN studentin s ON s.prid = pr.prid
JOIN belegung b ON s.prid = b.studentin_prid
JOIN kursbt kbt ON b.kursbt_kbtid = kbt.kbtid
JOIN kurs k ON kbt.kurs_kid = k.kid
JOIN semester sem ON k.semester_sid = sem.sid
JOIN termin t ON kbt.termin_tid = t.tid
JOIN wochentermin wt ON t.tid = wt.tid
JOIN tag tag ON wt.tag_tid = tag.tid
WHERE sem.beginn >= '2023-04-01' AND sem.ende <= '2023-09-30'
AND tag.bez = 'Montag';



In [12]:
ddbu.sql_print(con, sql)

┌─────────┬─────────┐
│  name   │ matrnr  │
│ varchar │ varchar │
├─────────┴─────────┤
│      0 rows       │
└───────────────────┘

